In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/", one_hot=False)

/home/apoorv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
import tensorflow as tf
# parameter
learning_rate=0.001
batch_size=120
num_steps=2000

In [3]:
# network parameter
num_input=784
num_classes=10
dropout=0.25

In [4]:
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        x=x_dict["images"]
        x=tf.reshape(x,shape=[-1,28,28,1])
        conv1=tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1=tf.layers.max_pooling2d(conv1, 2, 2)
        conv2=tf.layers.conv2d(conv1,64,3,activation=tf.nn.relu)
        conv2=tf.layers.max_pooling2d(conv2, 2, 2)
        fc1=tf.contrib.layers.flatten(conv2)
        fc1=tf.layers.dense(fc1,1024)
        fc1=tf.layers.dropout(fc1, rate= dropout, training=is_training )
        out=tf.layers.dense(fc1,n_classes)
    return(out)
        
        

In [5]:
def model_fn(features ,labels, mode):
    logits_train=conv_net(features,num_classes,dropout,reuse=False, is_training= True)
    logits_test=conv_net(features,num_classes,dropout,reuse=True, is_training=False )
    pred_classes=tf.argmax(logits_test, axis=1)
    pred_prob=tf.nn.softmax(logits_test)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    loss_op=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits
                           (logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op=optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    acc_op=tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [6]:
model=tf.estimator.Estimator(model_fn)
input_fn=tf.estimator.inputs.numpy_input_fn(x={'images':mnist.train.images}, y=mnist.train.labels, 
                                           batch_size=batch_size, num_epochs=20, shuffle=True)
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpv8srqgt_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8ee560add8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpv8srqgt_/model.ckpt.
INFO:ten

In [7]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-05-11:20:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpv8srqgt_/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-05-11:20:05
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9887, global_step = 2000, loss = 0.034726694
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpv8srqgt_/model.ckpt-2000
Testing Accuracy: 0.9887
